In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
dataset_path ="train.csv"
column_names = ['SalePrice', 'GrLivArea', 'YearBuilt']
dataset = pd.read_csv(dataset_path, usecols=column_names)
#学習データ準備
ss1 = StandardScaler()
ss2 = StandardScaler()
y = dataset['SalePrice']
X = dataset.loc[:, ['GrLivArea', 'YearBuilt']]
# y = np.array(y)
y = y.astype(np.int)[:, np.newaxis]
y = ss1.fit_transform(y)

# X = np.array(X)
X = ss2.fit_transform(X)

/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/DIC/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [3]:


# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.07
batch_size = 50
num_epochs = 500

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.square(Y - logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# variableの初期化
init = tf.global_variables_initializer()
with tf.name_scope('r2'):
    r2 = 1 - (tf.reduce_sum(tf.square(Y - logits)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))
    tf.summary.scalar('r2', r2)

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss,train_r2= sess.run([loss_op,r2], feed_dict={X: mini_batch_x, Y: mini_batch_y})
#             total_loss += loss
        total_loss /= total_batch
        val_loss,train_r2= sess.run([loss_op,r2], feed_dict={X: X_val, Y: y_val})
        print("Epoch",epoch,"loss :",loss," val_loss :",val_loss," r2 :",train_r2)

Instructions for updating:
Colocations handled automatically by placer.
Epoch 0 loss : 595.8972  val_loss : 529.3991  r2 : -607.2555
Epoch 1 loss : 40.210735  val_loss : 43.128487  r2 : -48.55267
Epoch 2 loss : 25.887337  val_loss : 19.75308  r2 : -21.69539
Epoch 3 loss : 6.275394  val_loss : 5.0515738  r2 : -4.8040285
Epoch 4 loss : 4.130791  val_loss : 3.5779943  r2 : -3.1109529
Epoch 5 loss : 3.0696335  val_loss : 2.1925857  r2 : -1.5191815
Epoch 6 loss : 2.585132  val_loss : 1.8074689  r2 : -1.0766997
Epoch 7 loss : 2.032832  val_loss : 1.4622049  r2 : -0.680007
Epoch 8 loss : 1.7417351  val_loss : 1.2835605  r2 : -0.47475266
Epoch 9 loss : 1.5650338  val_loss : 1.1898836  r2 : -0.36712217
Epoch 10 loss : 1.3578842  val_loss : 1.1102562  r2 : -0.27563393
Epoch 11 loss : 1.2550815  val_loss : 1.0518358  r2 : -0.20851147
Epoch 12 loss : 1.1399052  val_loss : 1.0101993  r2 : -0.16067314
Epoch 13 loss : 1.1019696  val_loss : 0.96313  r2 : -0.106592655
Epoch 14 loss : 0.9979415  val_los

Epoch 125 loss : 0.67377245  val_loss : 0.6697494  r2 : 0.23048836
Epoch 126 loss : 0.7469716  val_loss : 0.7585499  r2 : 0.12846059
Epoch 127 loss : 0.739903  val_loss : 0.7746117  r2 : 0.11000633
Epoch 128 loss : 0.8767397  val_loss : 0.84733564  r2 : 0.02644986
Epoch 129 loss : 0.86764205  val_loss : 0.84640163  r2 : 0.027522981
Epoch 130 loss : 0.82990134  val_loss : 0.8552145  r2 : 0.017397404
Epoch 131 loss : 0.83260113  val_loss : 0.8802513  r2 : -0.0113687515
Epoch 132 loss : 0.9108151  val_loss : 0.96857786  r2 : -0.11285198
Epoch 133 loss : 0.93378496  val_loss : 1.0148022  r2 : -0.16596174
Epoch 134 loss : 0.94544995  val_loss : 1.0448357  r2 : -0.20046878
Epoch 135 loss : 1.023091  val_loss : 1.1357915  r2 : -0.30497277
Epoch 136 loss : 1.0060526  val_loss : 1.0776895  r2 : -0.23821628
Epoch 137 loss : 1.1120092  val_loss : 1.1547315  r2 : -0.32673407
Epoch 138 loss : 0.81428784  val_loss : 0.91577566  r2 : -0.05218458
Epoch 139 loss : 1.1766933  val_loss : 1.244726  r2 : -

Epoch 252 loss : 0.22382563  val_loss : 0.32206842  r2 : 0.62995803
Epoch 253 loss : 0.22483419  val_loss : 0.32031327  r2 : 0.6319746
Epoch 254 loss : 0.22577694  val_loss : 0.31753635  r2 : 0.63516515
Epoch 255 loss : 0.22480494  val_loss : 0.31197333  r2 : 0.6415568
Epoch 256 loss : 0.2263542  val_loss : 0.31602433  r2 : 0.63690245
Epoch 257 loss : 0.22172338  val_loss : 0.308732  r2 : 0.64528096
Epoch 258 loss : 0.22743966  val_loss : 0.312977  r2 : 0.6404036
Epoch 259 loss : 0.22544834  val_loss : 0.30990884  r2 : 0.6439288
Epoch 260 loss : 0.22130434  val_loss : 0.30975908  r2 : 0.6441009
Epoch 261 loss : 0.23179142  val_loss : 0.3137778  r2 : 0.6394836
Epoch 262 loss : 0.2316889  val_loss : 0.3165902  r2 : 0.6362523
Epoch 263 loss : 0.23186195  val_loss : 0.31157243  r2 : 0.6420174
Epoch 264 loss : 0.23034166  val_loss : 0.31416485  r2 : 0.63903886
Epoch 265 loss : 0.2300437  val_loss : 0.3093597  r2 : 0.64455974
Epoch 266 loss : 0.23483649  val_loss : 0.31537005  r2 : 0.6376541

Epoch 376 loss : 0.1407342  val_loss : 0.23103063  r2 : 0.7345563
Epoch 377 loss : 0.13823923  val_loss : 0.23095389  r2 : 0.7346445
Epoch 378 loss : 0.14146453  val_loss : 0.22880358  r2 : 0.7371151
Epoch 379 loss : 0.13009684  val_loss : 0.22508875  r2 : 0.74138325
Epoch 380 loss : 0.13623594  val_loss : 0.22467403  r2 : 0.7418598
Epoch 381 loss : 0.15077603  val_loss : 0.26450148  r2 : 0.6960998
Epoch 382 loss : 0.13938527  val_loss : 0.26585573  r2 : 0.69454384
Epoch 383 loss : 0.13589114  val_loss : 0.23709457  r2 : 0.72758913
Epoch 384 loss : 0.1259224  val_loss : 0.23047157  r2 : 0.7351986
Epoch 385 loss : 0.13224503  val_loss : 0.22453842  r2 : 0.7420156
Epoch 386 loss : 0.1299011  val_loss : 0.2241444  r2 : 0.74246824
Epoch 387 loss : 0.13456841  val_loss : 0.2250791  r2 : 0.74139434
Epoch 388 loss : 0.133426  val_loss : 0.23219264  r2 : 0.7332212
Epoch 389 loss : 0.13478364  val_loss : 0.23001434  r2 : 0.735724
Epoch 390 loss : 0.13932605  val_loss : 0.24260853  r2 : 0.721253